https://catalog.data.gov/dataset/traffic-violations-56dda

## Traffic Violations
Metadata Updated: August 9, 2018

This dataset contains traffic violation information from all electronic traffic violations issued in the County. Any information that can be used to uniquely identify the vehicle, the vehicle owner or the officer issuing the violation will not be published.

Update Frequency: Daily


In [7]:
#import pandas as pd, holoviews as hv
import pandas as pd

import numpy as np

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.transform import factor_cmap
from bokeh.models import DatetimeTickFormatter
from bokeh.models.tools import HoverTool
from datetime import datetime as dt

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
df = pd.read_csv('/home/wfct0p/downloads/data/traffic-violation-small.csv', parse_dates=['Date Of Stop','Time Of Stop'])
df .head()

,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,Belts,...,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
0,2013-09-24,2019-03-28 17:11:00,MCP,"3rd district, Silver Spring",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,8804 FLOWER AVE,NaN,NaN,No,No,...,13-401(h),Transportation Article,No,BLACK,M,TAKOMA PARK,MD,MD,A - Marked Patrol,NaN
1,2017-08-29,2019-03-28 10:19:00,MCP,"2nd district, Bethesda",DRIVER FAILURE TO OBEY PROPERLY PLACED TRAFFIC...,WISCONSIN AVE@ ELM ST,38.981725,-77.092757,No,No,...,21-201(a1),Transportation Article,No,WHITE,F,FAIRFAX STATION,VA,VA,A - Marked Patrol,"(38.981725, -77.0927566666667)"
2,2014-12-01,2019-03-28 12:52:00,MCP,"6th district, Gaithersburg / Montgomery Village",FAILURE STOP AND YIELD AT THRU HWY,CHRISTOPHER AVE/MONTGOMERY VILLAGE AVE,39.162888,-77.229088,No,No,...,21-403(b),Transportation Article,No,BLACK,F,UPPER MARLBORO,MD,MD,A - Marked Patrol,"(39.1628883333333, -77.2290883333333)"
3,2017-08-29,2019-03-28 09:22:00,MCP,"3rd district, Silver Spring",FAILURE YIELD RIGHT OF WAY ON U TURN,CHERRY HILL RD./CALVERTON BLVD.,39.056975,-76.954633,No,No,...,21-402(b),Transportation Article,No,BLACK,M,FORT WASHINGTON,MD,MD,A - Marked Patrol,"(39.056975, -76.9546333333333)"
4,2017-08-28,2019-03-28 23:41:00,MCP,"6th district, Gaithersburg / Montgomery Village",FAILURE OF DR. TO MAKE LANE CHANGE TO AVAIL. L...,355 @ SOUTH WESTLAND DRIVE,NaN,NaN,No,No,...,21-405(e1),Transportation Article,No,WHITE,M,GAITHERSBURG,MD,MD,A - Marked Patrol,NaN


In [4]:
# Select violations by gender
ndf = pd.DataFrame() 
ndf['gender'] = df.Gender
ndf.index = df['Time Of Stop']
ndf.index.rename('time', inplace=True)

In [5]:
Fs = ndf.resample('h').sum()['gender'].apply(lambda x: x.count('F'))
Ms = ndf.resample('h').sum()['gender'].apply(lambda x: x.count('M'))

# Use bokeh structure
sourceF = ColumnDataSource(pd.DataFrame(Fs))
sourceM = ColumnDataSource(pd.DataFrame(Ms))

In [8]:
p = figure(x_axis_type='datetime', title="Traffic violation around the clock")
p.xaxis.formatter=DatetimeTickFormatter(days=["%H"])
   
# plot with lines
p.line(x='time', y='gender', source=sourceF, line_width=3, color='green', legend='Female')
p.line(x='time', y='gender', source=sourceM, line_width=3, color='blue', legend='Male')
p.legend.click_policy="hide"

p.add_tools(HoverTool(
    tooltips=[( 'time',   '@time{%Hh}'), ( 'total',  '@gender' )], 
    formatters={'time'      : 'datetime',}, 
    mode='vline')
           )

show(p)